<a id="top"></a>

    Just trying to figure out what's going on with all the data

# Import Packages

In [34]:
import pandas as pd
import dask.dataframe as dd
import datetime

***

- [Import Data Files](#import_data_files)
- [Trips Data](#trips_data)
- [Vehicles Data](#vehicles_data)
- [Leave Times Data](#leave_times_data)
- [Single Route Analysis](#single_route_analysis)
    


<a id="import_data_files"></a>
# Import Data Files

In [2]:
df_trips = pd.read_csv('/home/faye/data/rt_trips_DB_2018.txt', sep=';',error_bad_lines=False)
df_vehicles = pd.read_csv('/home/faye/data/rt_vehicles_DB_2018.txt', sep=';',error_bad_lines=False)
df_leavetimes = dd.read_csv('/home/faye/data/rt_leavetimes_DB_2018.txt', sep=';',error_bad_lines=False)

<a id="trips_data"></a>
# Trips Data

In [21]:
num_rows = df_trips.shape[0]
features = df_trips.shape[1]
print(f"The dataset has {num_rows} rows with {features} features.")

The dataset has 2182637 rows with 16 features.


In [23]:
df_trips.keys()

Index(['DATASOURCE', 'DAYOFSERVICE', 'TRIPID', 'LINEID', 'ROUTEID',
       'DIRECTION', 'PLANNEDTIME_ARR', 'PLANNEDTIME_DEP', 'ACTUALTIME_ARR',
       'ACTUALTIME_DEP', 'BASIN', 'TENDERLOT', 'SUPPRESSED', 'JUSTIFICATIONID',
       'LASTUPDATE', 'NOTE'],
      dtype='object')

In [24]:
df_trips.head(5)

,DATASOURCE,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,BASIN,TENDERLOT,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
0,DB,07-FEB-18 00:00:00,6253783,68,68_80,1,87245,84600,87524.0,84600.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2967409,"
1,DB,07-FEB-18 00:00:00,6262138,25B,25B_271,2,30517,26460,32752.0,NaN,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2580260,"
2,DB,07-FEB-18 00:00:00,6254942,45A,45A_70,2,35512,32100,36329.0,32082.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2448968,"
3,DB,07-FEB-18 00:00:00,6259460,25A,25A_273,1,57261,54420,58463.0,54443.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",3094242,"
4,DB,07-FEB-18 00:00:00,6253175,14,14_15,1,85383,81600,84682.0,81608.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2526331,"


- DATASOURCE: unique bus operator code
- DAYOFSERVICE: date
- TRIPID: unique trip code
- LINEID: unique line code
- ROUTEID: unique route code
- DIRECTION: binary var (1 or 2), inbound/going/northbound/eastbound OR outbound/back/southbound/westbound
- PLANNEDTIME_ARR: planned arrival time of the trip (seconds)
- PLANNEDTIME_DEP: planned departure time of the trip (seconds)
- ACTUALTIME_ARR: actual arrival time of the trip (seconds)
- ACTUALTIME_DEP: actual departure time of the trip (seconds)
- BASIN: basin code??
- TENDERLOT: tender lot??
- SUPPRESSED: the whole trip has been suppressed (0 = achieved, 1 = suppressed)
- JUSTIFICATIONID: fault code

In [26]:
df_trips['DAYOFSERVICE'].unique()

array(['07-FEB-18 00:00:00', '18-FEB-18 00:00:00', '16-MAR-18 00:00:00',
       '27-MAR-18 00:00:00', '11-MAR-18 00:00:00', '28-SEP-18 00:00:00',
       '26-SEP-18 00:00:00', '26-JUN-18 00:00:00', '21-JUN-18 00:00:00',
       '10-OCT-18 00:00:00', '11-OCT-18 00:00:00', '11-APR-18 00:00:00',
       '10-MAR-18 00:00:00', '20-SEP-18 00:00:00', '29-SEP-18 00:00:00',
       '22-JUN-18 00:00:00', '31-DEC-18 00:00:00', '07-JUL-18 00:00:00',
       '08-JUL-18 00:00:00', '24-SEP-18 00:00:00', '02-DEC-18 00:00:00',
       '09-JUL-18 00:00:00', '15-JUN-18 00:00:00', '16-OCT-18 00:00:00',
       '10-APR-18 00:00:00', '04-JUL-18 00:00:00', '12-APR-18 00:00:00',
       '30-DEC-18 00:00:00', '09-APR-18 00:00:00', '16-FEB-18 00:00:00',
       '21-FEB-18 00:00:00', '03-FEB-18 00:00:00', '12-FEB-18 00:00:00',
       '04-FEB-18 00:00:00', '22-MAR-18 00:00:00', '10-JUL-18 00:00:00',
       '20-APR-18 00:00:00', '19-JUN-18 00:00:00', '16-APR-18 00:00:00',
       '08-FEB-18 00:00:00', '28-OCT-18 00:00:00', 

In [27]:
df_trips['DIRECTION'].unique()

array([1, 2])

In [28]:
df_trips['BASIN'].unique()

array(['BasDef'], dtype=object)

In [29]:
df_trips['TENDERLOT'].unique()

array([nan])

In [34]:
df_trips['SUPPRESSED'].unique()

array([nan,  0.])

In [38]:
df_trips['SUPPRESSED'].value_counts()

0.0    4333
Name: SUPPRESSED, dtype: int64

- 4333 of the trips were achieved?, missing data for the rest

In [37]:
df_trips['JUSTIFICATIONID'].unique()

array([    nan, 194642., 205589., ..., 231826., 231813., 231765.])

 - BASIN and TENDERLOT seemed redundant
 - not sure whats going on with SUPPRESSED

<a id="vehicles_data"></a>
# Vehicles Data

In [10]:
num_rows = df_vehicles.shape[0]
features = df_vehicles.shape[1]
print(f"The dataset has {num_rows} rows with {features} features.")

The dataset has 272622 rows with 7 features.


In [12]:
df_vehicles.head(5)

,DATASOURCE,DAYOFSERVICE,VEHICLEID,DISTANCE,MINUTES,LASTUPDATE,NOTE
0,DB,23-NOV-18 00:00:00,3303848,286166,58849,04-DEC-18 08:03:09,NaN
1,DB,23-NOV-18 00:00:00,3303847,259545,56828,04-DEC-18 08:03:09,NaN
2,DB,28-FEB-18 00:00:00,2868329,103096,40967,08-MAR-18 10:35:59,NaN
3,DB,28-FEB-18 00:00:00,2868330,147277,43599,08-MAR-18 10:35:59,NaN
4,DB,28-FEB-18 00:00:00,2868331,224682,40447,08-MAR-18 10:35:59,NaN


- DISTANCE: distance travelled by the vehicle in this day
- MINUTES: time worked by the vehicle in the corresponding day

In [19]:
features = df_vehicles.keys()
for feature in features:
    print(feature)
    print()
    print(df_vehicles[feature].value_counts())

DATASOURCE

DB    272622
Name: DATASOURCE, dtype: int64
DAYOFSERVICE

04-MAY-18 00:00:00    911
09-MAR-18 00:00:00    909
23-NOV-18 00:00:00    908
13-APR-18 00:00:00    907
14-NOV-18 00:00:00    905
                     ... 
26-DEC-18 00:00:00    423
18-MAR-18 00:00:00    422
04-MAR-18 00:00:00    420
01-JAN-18 00:00:00    414
03-MAR-18 00:00:00    387
Name: DAYOFSERVICE, Length: 360, dtype: int64
VEHICLEID

2868382    342
2693284    342
2868404    341
2868354    339
2868334    337
          ... 
1001177      1
1000216      1
1001163      1
1001165      1
1000226      1
Name: VEHICLEID, Length: 1152, dtype: int64
DISTANCE

0         90
152449     9
154889     8
146750     8
178894     8
          ..
142901     1
144950     1
111357     1
149048     1
37778      1
Name: DISTANCE, Length: 170498, dtype: int64
MINUTES

0        42
62090    24
43560    24
61768    24
61503    23
         ..
12496     1
8402      1
6357      1
2263      1
4094      1
Name: MINUTES, Length: 57523, dtype: in

In [20]:
# Find number of duplicate rows
df = df_vehicles
num_duplicate_rows = df.duplicated().sum()
print(f"There are {num_duplicate_rows} duplicated rows in this dataset (excluding the first row).")
num_duplicate_rows_inclusive = df[df.duplicated(keep=False)].shape[0]
print(f"There are {num_duplicate_rows_inclusive} duplicated rows in this dataset (including row that is duplicated).")

There are 0 duplicated rows in this dataset (excluding the first row).
There are 0 duplicated rows in this dataset (including row that is duplicated).


<a id="leave_times_data"></a>
# Leave Times Data

In [3]:
# loaded it in with pandas, took awhile but worked? hope I haven't killed the server...
df_leavetimes = pd.read_csv('/home/faye/data/rt_leavetimes_DB_2018.txt', sep=';',error_bad_lines=False)

In [4]:
num_rows = df_leavetimes.shape[0]
features = df_leavetimes.shape[1]
print(f"The dataset has {num_rows} rows with {features} features.")

The dataset has 116949113 rows with 18 features.


In [5]:
df_leavetimes.keys()

Index(['DATASOURCE', 'DAYOFSERVICE', 'TRIPID', 'PROGRNUMBER', 'STOPPOINTID',
       'PLANNEDTIME_ARR', 'PLANNEDTIME_DEP', 'ACTUALTIME_ARR',
       'ACTUALTIME_DEP', 'VEHICLEID', 'PASSENGERS', 'PASSENGERSIN',
       'PASSENGERSOUT', 'DISTANCE', 'SUPPRESSED', 'JUSTIFICATIONID',
       'LASTUPDATE', 'NOTE'],
      dtype='object')

In [6]:
df_leavetimes.head(5)

,DATASOURCE,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,PASSENGERS,PASSENGERSIN,PASSENGERSOUT,DISTANCE,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
0,DB,01-JAN-18 00:00:00,5972116,12,119,48030,48030,48012,48012,2693211,NaN,NaN,NaN,NaN,NaN,NaN,08-JAN-18 17:21:10,NaN
1,DB,01-JAN-18 00:00:00,5966674,12,119,54001,54001,54023,54023,2693267,NaN,NaN,NaN,NaN,NaN,NaN,08-JAN-18 17:21:10,NaN
2,DB,01-JAN-18 00:00:00,5959105,12,119,60001,60001,59955,59955,2693263,NaN,NaN,NaN,NaN,NaN,NaN,08-JAN-18 17:21:10,NaN
3,DB,01-JAN-18 00:00:00,5966888,12,119,58801,58801,58771,58771,2693284,NaN,NaN,NaN,NaN,NaN,NaN,08-JAN-18 17:21:10,NaN
4,DB,01-JAN-18 00:00:00,5965960,12,119,56401,56401,56309,56323,2693209,NaN,NaN,NaN,NaN,NaN,NaN,08-JAN-18 17:21:10,NaN


In [7]:
df_leavetimes.describe().T

,count,mean,std,min,25%,50%,75%,max
TRIPID,116949113.0,7.150967e+06,7.323270e+05,5955221.0,6.592950e+06,7.110714e+06,8.018861e+06,8.592207e+06
PROGRNUMBER,116949113.0,3.117937e+01,1.997076e+01,1.0,1.500000e+01,2.900000e+01,4.500000e+01,1.090000e+02
STOPPOINTID,116949113.0,2.500256e+03,2.016848e+03,2.0,9.250000e+02,1.959000e+03,3.708000e+03,7.692000e+03
PLANNEDTIME_ARR,116949113.0,5.326348e+04,1.749662e+04,16200.0,3.797500e+04,5.328900e+04,6.724300e+04,9.168000e+04
PLANNEDTIME_DEP,116949113.0,5.326348e+04,1.749662e+04,16200.0,3.797500e+04,5.328900e+04,6.724300e+04,9.168000e+04
ACTUALTIME_ARR,116949113.0,5.344637e+04,1.750710e+04,15974.0,3.810500e+04,5.351100e+04,6.750200e+04,9.717700e+04
ACTUALTIME_DEP,116949113.0,5.345900e+04,1.750640e+04,15974.0,3.811700e+04,5.352600e+04,6.751400e+04,9.717700e+04
VEHICLEID,116949113.0,2.011279e+06,8.027830e+05,1000110.0,1.001087e+06,2.172293e+06,2.693244e+06,3.394131e+06
PASSENGERS,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PASSENGERSIN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- DAYOFSERVICE: date
- TRIPID: unique trip code - there will be multiple trips in one route so it is used to differentiate each trip
- PROGRNUMBER: sequential position of the stop point in the trip - shows how far a specific route has gone
- STOPPOINTID: unique stop point code - might be number on the stop?
- PLANNEDTIME_ARR: planned arrival time at the stop point (seconds)
- PLANNEDTIME_DEP: planned departure time from the stop point (seconds)
- ACTUALTIME_ARR: actual arrival time at the stop point (seconds)
- ACTUALTIME_DEP: actual departure time from the stop point (seconds)
- VEHICLEID: unique vehicle code arriving at this stop point
- PASSENGERS: num of passengers on board
- PASSENGERSIN: num of boarded passengers
- PASSENGERSOUT: num of descended passengers
- DISTANCE: distance measured from the beginning of the trip
- SUPPRESSED: when the trip is partially suppressed it says that the previous link is suppressed (0 = achieved, 1 = suppressed)
- JUSTIFICATIONID: fault code

In [49]:
df_leavetimes['DATASOURCE'].unique()

array(['DB'], dtype=object)

In [51]:
df_leavetimes['SUPPRESSED'].unique()

array([nan,  1.,  0.])

In [52]:
df_leavetimes['SUPPRESSED'].value_counts()

0.0    438942
1.0    149718
Name: SUPPRESSED, dtype: int64

In [53]:
df_leavetimes['NOTE'].unique()

array([nan])

<a id="single_route_analysis"></a>
# Single Route Analysis

    Going to take a look at a single route.

In [20]:
df_trips.head(3)

,DATASOURCE,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,BASIN,TENDERLOT,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
0,DB,07-FEB-18 00:00:00,6253783,68,68_80,1,87245,84600,87524.0,84600.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2967409,"
1,DB,07-FEB-18 00:00:00,6262138,25B,25B_271,2,30517,26460,32752.0,NaN,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2580260,"
2,DB,07-FEB-18 00:00:00,6254942,45A,45A_70,2,35512,32100,36329.0,32082.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2448968,"


In [21]:
df_vehicles.head(3)

,DATASOURCE,DAYOFSERVICE,VEHICLEID,DISTANCE,MINUTES,LASTUPDATE,NOTE
0,DB,23-NOV-18 00:00:00,3303848,286166,58849,04-DEC-18 08:03:09,NaN
1,DB,23-NOV-18 00:00:00,3303847,259545,56828,04-DEC-18 08:03:09,NaN
2,DB,28-FEB-18 00:00:00,2868329,103096,40967,08-MAR-18 10:35:59,NaN


In [22]:
df_leavetimes.head(3)

,DATASOURCE,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,PASSENGERS,PASSENGERSIN,PASSENGERSOUT,DISTANCE,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
0,DB,01-JAN-18 00:00:00,5972116,12,119,48030,48030,48012,48012,2693211,NaN,NaN,NaN,NaN,NaN,NaN,08-JAN-18 17:21:10,NaN
1,DB,01-JAN-18 00:00:00,5966674,12,119,54001,54001,54023,54023,2693267,NaN,NaN,NaN,NaN,NaN,NaN,08-JAN-18 17:21:10,NaN
2,DB,01-JAN-18 00:00:00,5959105,12,119,60001,60001,59955,59955,2693263,NaN,NaN,NaN,NaN,NaN,NaN,08-JAN-18 17:21:10,NaN


## The Trip

    Going to go with trip 6262138

In [26]:
# make a new df of entries with only that trip id
df_single_route = df_trips[df_trips['TRIPID'] == 6262138]
df_single_route

,DATASOURCE,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,BASIN,TENDERLOT,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
1,DB,07-FEB-18 00:00:00,6262138,25B,25B_271,2,30517,26460,32752.0,NaN,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2580260,"
140264,DB,15-FEB-18 00:00:00,6262138,25B,25B_271,2,30517,26460,30016.0,NaN,BasDef,NaN,NaN,NaN,28-FEB-18 13:51:50,",2580260,"
222674,DB,16-FEB-18 00:00:00,6262138,25B,25B_271,2,30517,26460,30117.0,26382.0,BasDef,NaN,NaN,NaN,26-FEB-18 12:19:30,",2580260,"
562075,DB,14-FEB-18 00:00:00,6262138,25B,25B_271,2,30517,26460,30580.0,26578.0,BasDef,NaN,NaN,NaN,15-MAR-18 12:47:59,",2580260,"
1057230,DB,08-FEB-18 00:00:00,6262138,25B,25B_271,2,30517,26460,30475.0,26373.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:20:01,",2580260,"
1192237,DB,05-FEB-18 00:00:00,6262138,25B,25B_271,2,30517,26460,30796.0,26432.0,BasDef,NaN,NaN,NaN,28-FEB-18 11:29:31,",2580260,"
1310647,DB,09-FEB-18 00:00:00,6262138,25B,25B_271,2,30517,26460,30922.0,NaN,BasDef,NaN,NaN,NaN,28-FEB-18 12:32:51,",2580260,"
1554930,DB,12-FEB-18 00:00:00,6262138,25B,25B_271,2,30517,26460,30587.0,25962.0,BasDef,NaN,NaN,NaN,28-FEB-18 13:18:29,",2580260,"
1578863,DB,13-FEB-18 00:00:00,6262138,25B,25B_271,2,30517,26460,30532.0,26422.0,BasDef,NaN,NaN,NaN,28-FEB-18 13:29:11,",2580260,"
1615634,DB,06-FEB-18 00:00:00,6262138,25B,25B_271,2,30517,26460,31289.0,26653.0,BasDef,NaN,NaN,NaN,28-FEB-18 11:48:21,",2580260,"


    Turns out the 'unique trip code' is not so unique?
    I'll go with one day from that selection, the 14th <3

In [29]:
# remove all other dates except 14th feb from the df
df_single_route = df_single_route[df_single_route['DAYOFSERVICE'] == '14-FEB-18 00:00:00']
df_single_route

,DATASOURCE,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,BASIN,TENDERLOT,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
562075,DB,14-FEB-18 00:00:00,6262138,25B,25B_271,2,30517,26460,30580.0,26578.0,BasDef,NaN,NaN,NaN,15-MAR-18 12:47:59,",2580260,"


In [33]:
# drop seemingly redundant columns
redundant_columns = ['DATASOURCE', 'BASIN', 'TENDERLOT', 'JUSTIFICATIONID', 'LASTUPDATE']
df_single_route = df_single_route.drop(columns=redundant_columns)
df_single_route

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,SUPPRESSED,NOTE
562075,14-FEB-18 00:00:00,6262138,25B,25B_271,2,30517,26460,30580.0,26578.0,NaN,",2580260,"


### Convert the Arr and Dep times to HH:MM:SS

In [36]:
str(datetime.timedelta(seconds=30517))

'8:28:37'

In [38]:
# convert the time for planned arrival time (testing it works)
for time in df_single_route['PLANNEDTIME_ARR']:
    #print(time)
    df_single_route['PLANNEDTIME_ARR'] = str(datetime.timedelta(seconds=time))

In [39]:
df_single_route

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,SUPPRESSED,NOTE
562075,14-FEB-18 00:00:00,6262138,25B,25B_271,2,8:28:37,26460,30580.0,26578.0,NaN,",2580260,"


In [40]:
# convert the other times
for time in df_single_route['PLANNEDTIME_DEP']:
    #print(time)
    df_single_route['PLANNEDTIME_DEP'] = str(datetime.timedelta(seconds=time))

for time in df_single_route['ACTUALTIME_ARR']:
    #print(time)
    df_single_route['ACTUALTIME_ARR'] = str(datetime.timedelta(seconds=time))
    
for time in df_single_route['ACTUALTIME_DEP']:
    #print(time)
    df_single_route['ACTUALTIME_DEP'] = str(datetime.timedelta(seconds=time))

In [41]:
df_single_route

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,SUPPRESSED,NOTE
562075,14-FEB-18 00:00:00,6262138,25B,25B_271,2,8:28:37,7:21:00,8:29:40,7:22:58,NaN,",2580260,"


In [43]:
cols = ['DAYOFSERVICE', 'TRIPID', 'LINEID', 'ROUTEID', 'DIRECTION',
       'PLANNEDTIME_ARR','ACTUALTIME_ARR', 'PLANNEDTIME_DEP', 
       'ACTUALTIME_DEP', 'SUPPRESSED', 'NOTE']

In [45]:
df_single_route[cols]

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,ACTUALTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_DEP,SUPPRESSED,NOTE
562075,14-FEB-18 00:00:00,6262138,25B,25B_271,2,8:28:37,8:29:40,7:21:00,7:22:58,NaN,",2580260,"


    So it arrives at 08:29 and departs at 07:22? Or does it depart at 07:22 and arrive at 08:29? Which comes first?

## The Vehicle

In [47]:
df_vehicles[df_vehicles['DAYOFSERVICE'] == '14-FEB-18 00:00:00']

,DATASOURCE,DAYOFSERVICE,VEHICLEID,DISTANCE,MINUTES,LASTUPDATE,NOTE
50478,DB,14-FEB-18 00:00:00,1000110,114978,43057,15-MAR-18 12:47:59,NaN
50479,DB,14-FEB-18 00:00:00,1000111,28144,3787,15-MAR-18 12:47:59,NaN
50480,DB,14-FEB-18 00:00:00,1000113,266321,52991,15-MAR-18 12:47:59,NaN
50481,DB,14-FEB-18 00:00:00,1000115,344083,64749,15-MAR-18 12:47:59,NaN
50482,DB,14-FEB-18 00:00:00,1000116,140370,26511,15-MAR-18 12:47:59,NaN
...,...,...,...,...,...,...,...
56117,DB,14-FEB-18 00:00:00,3088366,222229,40014,15-MAR-18 12:47:59,NaN
56118,DB,14-FEB-18 00:00:00,3088367,71426,6073,15-MAR-18 12:47:59,NaN
56119,DB,14-FEB-18 00:00:00,3088368,32868,4506,15-MAR-18 12:47:59,NaN
56120,DB,14-FEB-18 00:00:00,3088373,236856,21669,15-MAR-18 12:47:59,NaN


## The Leave Times

In [51]:
# make a new df of entries with only that trip id
df_single_route_leavetimes = df_leavetimes[df_leavetimes['TRIPID'] == 6262138]
df_single_route_leavetimes = df_single_route_leavetimes[df_single_route_leavetimes['DAYOFSERVICE'] == '14-FEB-18 00:00:00']
df_single_route_leavetimes

,DATASOURCE,DAYOFSERVICE,TRIPID,PROGRNUMBER,STOPPOINTID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,VEHICLEID,PASSENGERS,PASSENGERSIN,PASSENGERSOUT,DISTANCE,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
18558367,DB,14-FEB-18 00:00:00,6262138,6,3368,26988,26988,26930,26952,1001208,NaN,NaN,NaN,NaN,NaN,NaN,15-MAR-18 12:47:59,NaN
18558402,DB,14-FEB-18 00:00:00,6262138,37,406,30212,30212,30194,30218,1001208,NaN,NaN,NaN,NaN,NaN,NaN,15-MAR-18 12:47:59,NaN
18558437,DB,14-FEB-18 00:00:00,6262138,38,4530,30343,30343,30340,30354,1001208,NaN,NaN,NaN,NaN,NaN,NaN,15-MAR-18 12:47:59,NaN
18563006,DB,14-FEB-18 00:00:00,6262138,7,7137,27107,27107,27053,27082,1001208,NaN,NaN,NaN,NaN,NaN,NaN,15-MAR-18 12:47:59,NaN
18563118,DB,14-FEB-18 00:00:00,6262138,40,7390,30517,30517,30580,30580,1001208,NaN,NaN,NaN,NaN,NaN,NaN,15-MAR-18 12:47:59,NaN
18567705,DB,14-FEB-18 00:00:00,6262138,10,4619,27467,27467,27324,27324,1001208,NaN,NaN,NaN,NaN,NaN,NaN,15-MAR-18 12:47:59,NaN
18572362,DB,14-FEB-18 00:00:00,6262138,12,4629,27591,27591,27440,27440,1001208,NaN,NaN,NaN,NaN,NaN,NaN,15-MAR-18 12:47:59,NaN
18577066,DB,14-FEB-18 00:00:00,6262138,14,4631,27748,27748,27578,27601,1001208,NaN,NaN,NaN,NaN,NaN,NaN,15-MAR-18 12:47:59,NaN
18581809,DB,14-FEB-18 00:00:00,6262138,16,5164,27866,27866,27770,27770,1001208,NaN,NaN,NaN,NaN,NaN,NaN,15-MAR-18 12:47:59,NaN
18586557,DB,14-FEB-18 00:00:00,6262138,18,2234,28142,28142,27982,28015,1001208,NaN,NaN,NaN,NaN,NaN,NaN,15-MAR-18 12:47:59,NaN


In [53]:
# only going to keep these columns
cols = ['DAYOFSERVICE', 'TRIPID', 'PROGRNUMBER', 'STOPPOINTID',
       'PLANNEDTIME_ARR','ACTUALTIME_ARR', 'PLANNEDTIME_DEP', 
       'ACTUALTIME_DEP', 'VEHICLEID']

# removing all the passenger featues as there is nothing there for this instance

In [ ]:
df_single_route_leavetimes = df

***

[Back to top](#top)